In [6]:
from selenium import webdriver
import time 
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import pandas as pd

In [ ]:
driver = webdriver.Chrome(executable_path = 'C:/Users/Dreambuilds/Downloads/webdriver/chomedriver.exe')
wait = WebDriverWait(driver, 10) #wait 10s to let webpage load

driver.get('https://clinicaltrials.gov/ct2/about-site/crawling') #getting website url
driver.maximize_window()
time.sleep(1)

#creating dict to store data
clinical_trial = {}
last_date_list = []
summary_list = []
description_list = []
condition_list = []
intervention_list = []
criteria_list = []
gender_list = []
age_list = []

# each row in first page
for x in range(2, 193): 
    #each column in first page
    for y in range(1,4):
        #index for the 5 links in each square
        for z in range(1,6):

            xpath_1 = '//*[@id="body-copy"]/div[1]/table/tbody/tr[{}]/td[{}]/a[{}]'.format(x,y,z)
            page_1 = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_1))) 
            page_1.click()

            try:
                #row in second page                                                
                for a in range(2,70):
                    #column in second page                                       
                    for b in range(1,6):
                        #index for the 5 links in each square
                        for c in range(1,6):

                            try:
                                xpath_2 = '//*[@id="body-copy"]/table/tbody/tr[{}]/td[{}]/a[{}]'.format(a,b,c)
                                trial = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_2)))
                                trial.click()

                            except:
                                driver.quit()

                            time.sleep(1) #delay to let page load

                            #clicking on 'Tabular view' button
                            tabular_view = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="tabular"]/a')))
                            tabular_view.click()

                            #scrolling page to allow for faster scraping
                            for i in range(1, 11):
                                driver.execute_script("window.scrollBy(0,300)")
                                time.sleep(0.5)

                            #getting last updated date
                            #checking if header contains "Last Update Posted Date"
                            last_update_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Last Update Posted Date')]")))

                            last_update = last_update_header.find_element(By.XPATH, "./following-sibling::td") #getting td element within the header
                            last_date_list.append(last_update.text) #adding the text to the list

                            #getting brief summary
                            #checking if header contains "Brief Summary"
                            brief_summary_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Brief Summary')]")))

                            brief_summary = brief_summary_header.find_element(By.XPATH, './following-sibling::td') #getting td element within the header
                            summary_list.append(brief_summary.text) #adding the text to the list

                            #getting detailed description
                            #checking if header contains "Detailed Description"
                            detail_description_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Detailed Description')]")))

                            detail_description = detail_description_header.find_element(By.XPATH, './following-sibling::td') #getting td element within the header
                            description_list.append(detail_description.text) #adding the text to the list

                            #Condition of the trial
                            # Check if the header contains "Condition"
                            header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Condition')]")))

                            condition = header.find_element(By.XPATH, "./following-sibling::td") #getting td element within the header
                            condition_list.append(condition.text) #adding the text to the list

                            #intervention of trial
                            #checking if header contains "header"
                            intervention_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Intervention')]")))

                            intervention = intervention_header.find_element(By.XPATH, './following-sibling::td') #getting td element within the header
                            intervention_list.append(intervention.text) #adding the text to the list


                            #Eligibility Criteria
                            #checking if header contains "Eligibility Criteria"
                            criteria_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Eligibility Criteria')]")))

                            criteria = criteria_header.find_element(By.XPATH, './/following-sibling::td') #getting td element within the header
                            criteria_list.append(criteria.text) #adding the text to the list

                            #Gender
                            #checking if header contains "Sex/Gender"
                            gender_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Sex/Gender')]")))

                            gender = gender_header.find_element(By.XPATH, ".//following-sibling::td") #getting td element within the header
                            gender_list.append(gender.text) #adding the text to the list


                            #Age
                            #checking if header contains "Ages"
                            age_header = wait.until(EC.visibility_of_element_located((By.XPATH, ".//th[contains(text(), 'Ages')]")))

                            age = age_header.find_element(By.XPATH, './/following-sibling::td') #getting td element within the header
                            age_list.append(age.text) #adding the text to the list

                            driver.back() #return to study detail
                            time.sleep(1)
                            driver.back() #return to previous page

            except:
                driver.quit() #end program if error faced

        
                
           
                
#creating the key pairs in the dict
clinical_trial["Last Updated"] = last_date_list
clinical_trial["Summary"] = summary_list
clinical_trial["Description"] = description_list
clinical_trial["Condition"] = condition_list
clinical_trial["Intervention"] = intervention_list
clinical_trial["Criteria"] = criteria_list
clinical_trial["Gender"] = gender_list
clinical_trial["Age"] = age_list

   
    

<ipython-input-7-a7da4f5823db>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:/Users/Dreambuilds/Downloads/webdriver/chomedriver.exe')


In [ ]:
#creating dataframe of data collected
df = pd.DataFrame(clinical_trial) 
df.head(10)

In [ ]:
#converting dataframe to csv file
df.to_csv('C:/Users/Dreambuilds/Downloads/clinical_trial.csv', index = False)